In [1]:
import mysql.connector
from nltk.tokenize import sent_tokenize
import import_ipynb
import re
from Sastrawi.Stemmer.StemmerFactory import StemmerFactory
import pandas as pd
from algoritma.MMR import *
from nltk.corpus import stopwords
import string
from termcolor import colored
import time
import csv
from nltk.tokenize import word_tokenize

importing Jupyter notebook from C:\Users\risma\Kuliah\MCA\Text-Summarization-News-Ariticles\algoritma\MMR.ipynb


# Deklarasi Connection Database

Membuat koneksi kedatabase lokal dengan database 'db-berita-mca'

In [20]:
mydb = mysql.connector.connect(
    host="localhost",
    user="root",
    password="",
    database="db-berita-mca"  
) 

# Membuat Fungsi Steaming Use Sastrawi

Membuat stemmer pakai sastrawi 
Digunakan untuk melakukan steaming kalimat (menjadi kalimat dasar Menggunakan Library Sastrawi)

In [3]:
factory = StemmerFactory()
stemmer = factory.create_stemmer()

# Load Data Stopword Lib & File Internal

Mengambil data stopword bahasa indonesia yang pertama menggunakan file list dari stop word 917 dan menggunakan library nltk sebanyak panjang data 789

In [4]:
def load_stopWords():
    r = open("stopwordlist-indonesia.txt", "r")
    readTxt = r.read()
    idStopWord = readTxt.split() 
    return idStopWord

In [5]:
def load_stopWordsNLTK():
    stop_words = stopwords.words('indonesian')
    return stop_words

# Simpan hasil summary ke database

Menyimpan data yang telah dihasilakan melalui summary ke database dengan cara mengupdate data tersebut dan memasukan text summary pada field tersebut

In [6]:
def simpanData(mydb,summary,id,labeling_summary,score_summary,labeling_content,score_content):
    mycursor = mydb.cursor()
    id = int(id)
    sql = "UPDATE tb_berita SET summary = %s ,labeling_summary = %s ,score_summary = %s,labeling_content = %s,score_content = %s WHERE id = %s"
    param = (summary,labeling_summary,score_summary,labeling_content,score_content,id)
    mycursor.execute(sql, param)
    mydb.commit()

# Proses Text Preprocessing 

[tokenizeParagraftoKalimat, cleansingData, steamingData, indoStopWord]

Fungsi diatas merupakan fungsi yang digunakan untuk melakukan preprocessing article sebuah berita, penjelasan mengenai fungsi akan dijelaskan lebih lanjut

[tokenizeParagraftoKalimat]

berfungsi sebagai tokenize pargraf menjadi bagian perkalimat 

In [7]:
def tokenizeParagraftoKalimat(paragraf):
    tokenize = sent_tokenize(paragraf)
    return tokenize

[cleansingData]

berfungsi sebagai membersihkan data dari huruf tanda petik dan masih kata besar atau keceil

In [8]:
def cleansingData(text):
    hasil = re.sub('[^A-Za-z0-9 ]+', '', text)
    hasil = hasil.translate(str.maketrans("","",string.punctuation))
    return hasil

[steamingData]

berguna sebagai menjadikan kata dasar sebuah kalimat dan fungsi stemmer tersebut sekaligus menjadikan kalimat menjadi lower atau huruf kecil

In [9]:
def steamingData(text):
    steamingData = stemmer.stem(text)
    return steamingData

[indoStopWord]

berguna dalam menghilangkan kata sambung atau tidak diperlukan, ini berfungsi dlam menghitung kesamaan antar kalimat pada algoritma MMR agar lebih mudah dan mendapatkan hasil yang akurat (Lib NLTK & List Stopword)

In [10]:
def indoStopWord(stopNLTK, stopTxt, kalimat):
    hasilStopWord = []
    for kata in kalimat.split():
        if not kata in stopNLTK:
            if not kata in stopTxt:
                hasilStopWord.append(kata)
    return " ".join(hasilStopWord)

[hasilSummary]

fungsi yang berguna dalam menampilkan hasil summarization pada algoritma MMR

In [11]:
def hasilSummary(kalimatAsli, summarySet):
    hasil = []
    for sentence in summarySet:
        hasil.append(kalimatAsli [sentence].lstrip(' ')) 
    return " ".join(hasil)

[visualisasiSummaryOri]

menampilkan visualisasi yang sangat berguna dalam mengetahui kata mana saja yang dipakai atau ditampilkan pada summarization

In [12]:
def visualisasiSummaryOri(cleansingTeks,summarySet,kalimatAsli):
    for sentence in cleansingTeks:
        if sentence in summarySet:
            print(colored(kalimatAsli[sentence].lstrip(' '),'red'))
        else:
            print(kalimatAsli[sentence].lstrip(' '))

bisa dibilang sebagai code utama, dimana disini tempat menjalankan semua code dengan melakukan pemanggilan semua fungsi dan class algoritam MMR 

In [58]:
def summarization (data):
    for iteration, article in enumerate(data['article']):
        id = df['id'][iteration]
        tokenizeKalimat = tokenizeParagraftoKalimat(article)

        kalimat = []
        cleansingTeks = []
        kalimatAsli = {}

        for teks in tokenizeKalimat:
            kataDasar = steamingData(cleansingData(teks))
            dataFinal = indoStopWord(load_stopWordsNLTK(),load_stopWords(),kataDasar)
            cleansingTeks.append(dataFinal)
            kalimat.append(teks)
            kalimatAsli[dataFinal] = teks

        iteration = MMR()
        score = iteration.similarityScore(cleansingTeks)
        summarySet = iteration.calculateMMR(score,cleansingTeks)


        summary = hasilSummary(kalimatAsli,summarySet)
        polaritySummary,scoreSummary = analisisSummary(summary)
        content = " ".join(cleansingTeks)
        polarityContent,scoreContent = analisisSummary(content)
        simpanData(mydb,summary,id,polaritySummary,scoreSummary,polarityContent,scoreContent)
        print ('\nSummary dari Berita ID : {} (hasil teks yang diringkas):\n'.format(id))
        print(summary)
        print ('\nAnalisis Sentiment :'+polaritySummary)
        print ('\nDengan Score Sentiment :'+str(scoreSummary))  

        print ('=============================================================')
        print ('\Oringinal Article (Teks Asli):\n')
        visualisasiSummaryOri(cleansingTeks,summarySet,kalimatAsli)
        

[analisisSummary] fungsi yang berguna untuk melakukan analysis sentiment menggunkaan model lexicon da

In [59]:
def analisisSummary(text):
    lexicon_positive = dict()
    with open('data/lexicon_positive.csv', 'r') as csvfile:
        reader = csv.reader(csvfile, delimiter=',')
        for row in reader:
            lexicon_positive[row[0]] = int(row[1])
        
    lexicon_negative = dict()
    with open('data/lexicon_negative.csv', 'r') as csvfile:
        reader = csv.reader(csvfile, delimiter=',')
        for row in reader:
            lexicon_negative[row[0]] = int(row[1])
            
    tokonize = (word_tokenize(cleansingData(text)))
    hasilStopWord = []
    for kata in tokonize:
        if not kata in load_stopWordsNLTK():
            if not kata in load_stopWords():
                hasilStopWord.append(kata)
    hasilSteamingData = []
    for x in hasilStopWord:
        hasilSteamingData.append(steamingData(x))
    score = 0
    for word in hasilSteamingData:
        if (word in lexicon_positive):
            score = score + lexicon_positive[word]
    for word in hasilStopWord:
         if (word in lexicon_negative):
            score = score + lexicon_negative[word]
    polarity=''
    if (score > 0):
        polarity = 'positive'
    elif (score < 0):
        polarity = 'negative'
    else:
        polarity = 'neutral'
        
    return polarity,score

# Mengambil Data pada Database

Mengambil data pada database,data yang diambil hanya 2 kolom saja

In [60]:
mycursor = mydb.cursor()
mycursor.execute("SELECT id, content FROM tb_berita WHERE summary IS NULL")
myresult = mycursor.fetchall()
df = pd.DataFrame(myresult,columns=['id','article'])
df

,id,article
0,2439,"Jakarta, CNBC Indonesia - Infeksi virus corona..."


In [61]:
summarization(df)


Summary dari Berita ID : 2439 (hasil teks yang diringkas):

Jakarta, CNBC Indonesia - Infeksi virus corona di Korea Selatan (Korsel) mencatat lebih dari 7.000 kasus selama tiga hari berturut-turut, Jumat (10/12/2021). Pejabat juga mengeluarkan perintah administratif yang mengharuskan rumah sakit di seluruh negeri mempersiapkan 2.000 atau lebih banyak tempat tidur yang digabungkan untuk perawatan Covid-19. Sekitar 41,5 juta orang, atau 81% dari populasi lebih dari 51 juta, telah divaksinasi lengkap, tetapi hanya 10% yang menerima suntikan booster.

Analisis Sentiment :positive

Dengan Score Sentiment :7
\Oringinal Article (Teks Asli):

Jakarta, CNBC Indonesia - Infeksi virus corona di Korea Selatan (Korsel) mencatat lebih dari 7.000 kasus selama tiga hari berturut-turut, Jumat (10/12/2021).
Jumlah ini memecahkan rekor dan membuat rumah sakit kelimpungan.
Perdana Menteri Korsel Kim Boo-kyum mengatakan Korsel bisa mengambil tindakan luar biasa jika kenaikan kasus tidak segera melambat.
"